In [116]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from matplotlib import pyplot as plt
import seaborn as sns
import vccfcoords
import math
import localization as lx
import os, sys
from typing import Dict, Any
import hashlib
import json

In [157]:
def sim_coordinates(n, ndim, min=0, max=1):

    val_range = max - min
    generated_coordinates = np.random.rand(n, ndim) * val_range + min
    #coord_dict = {x:generated_coordinates[x] for x in list(range(n))}
    
    return generated_coordinates

def get_euclid_dist(pt1, pt2):

    return distance.euclidean(pt1, pt2)

def get_closest_points(coords, target, n):

    distances = distance.cdist([target], coords, metric='euclidean')[0]

    closest_idx = np.argsort(distances)[:n]
    closest_distances = distances[closest_idx]

    return closest_idx, closest_distances
    
def dict_to_array(coord_dict):

    return np.vstack(list(coord_dict.values()))


def plot_coordinates(coords, ndim):

    assert(ndim in [2,3])

    #coords = dict_to_array(coord_dict)

    if ndim == 2:
        x_vals = coords[:, 0:1]
        y_vals = coords[:, 1:2]

        plt.scatter(x_vals, y_vals)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.show()

        return
    
    else:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Generate the values
        x_vals = coords[:, 0:1]
        y_vals = coords[:, 1:2]
        z_vals = coords[:, 2:3]

        # Plot the values
        ax.scatter(x_vals, y_vals, z_vals, c = 'b', marker='o')
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')

        plt.show()

        return

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def estimate_coord(coords, distances):
    
    dim = coords.shape[1]
    dim_str = str(dim) + "D"

    state = lx.Project(mode=dim_str,solver='LSE')

    for coord_idx in range(len(coords)):
        state.add_anchor(str(coord_idx), tuple(coords[coord_idx]))
    
    unknown_cell, unkown_cell_label= state.add_target()

    for dist_idx in range(len(distances)):
        unknown_cell.add_measure(str(dist_idx),distances[dist_idx])

    with HiddenPrints():
        state.solve()

    computed_coord = None

    if dim == 2:
        computed_coord = np.array((unknown_cell.loc.x, unknown_cell.loc.y))
    else:
        computed_coord = np.array((unknown_cell.loc.x, unknown_cell.loc.y, unknown_cell.loc.z))

    return computed_coord


def estimate_coords(EC_coords, closest_cells_dict):

    COORD_IDX = 0
    DIST_IDX = 1

    computed_coords = dict()

    for key,val in closest_cells_dict.items():
        reference_coords = EC_coords[val[COORD_IDX]]
        reference_dists = val[DIST_IDX]

        computed_coord = estimate_coord(reference_coords, reference_dists)

        computed_coords[key] = computed_coord

    return computed_coords

def get_avg_dist_error(true_coords, pred_coords_dict):

    dist_error = [get_euclid_dist(true_coords[x], pred_coords_dict[x]) for x in pred_coords_dict]

    return np.mean(dist_error)

def run_simulation(n_EC,
                   n_other_cells,
                   n_dim,
                   n_closest,
                   coord_min,
                   coord_max):
    
    COORD_IDX = 0
    DIST_IDX = 1
    
    #Generating Coordinates
    EC_coords = sim_coordinates(n_EC, n_dim, coord_min, coord_max)
    other_cell_coords = sim_coordinates(n_other_cells, n_dim, coord_min, coord_max)

    simulated_coords_hash = hash(np.vstack((EC_coords, other_cell_coords)).tobytes())

    n_closest_list = n_closest if isinstance(n_closest, list) else [n_closest]

    closest_EC_cells_dict = {x:get_closest_points(EC_coords,other_cell_coords[x], n_other_cells) for x in list(range(len(other_cell_coords)))}

    avg_dist_error = []

    #Using Multilateration engine to estimate coordinates given distances to endothelial cells
    for n_closest_num in n_closest_list:
        closest_EC_cells_dict_subset = {x:(y[COORD_IDX][:n_closest_num],y[DIST_IDX][:n_closest_num]) for x,y in closest_EC_cells_dict.items()}
        computed_coords = estimate_coords(EC_coords, closest_EC_cells_dict_subset)
        avg_dist_error = get_avg_dist_error(other_cell_coords, computed_coords)

        avg_dist_error.append(avg_dist_error)

    avg_dist_error = get_avg_dist_error(other_cell_coords, computed_coords

    avg_dist_error = get_avg_dist_error(other_cell_coords, computed_coords)

    experiment_dict = dict()
    experiment_dict["simulated_coords_hash"] = simulated_coords_hash
    experiment_dict["n_EC"] = n_EC
    experiment_dict["n_other_cells"] = n_other_cells
    experiment_dict["n_dim"] = n_dim
    experiment_dict["coord_min"] = coord_min
    experiment_dict["coord_max"] = coord_max
    experiment_dict["n_closest"] = n_closest
    experiment_dict["avg_dist_error"] = avg_dist_error

    return experiment_dict

In [ ]:

    #Using Multilateration engine to estimate coordinates given distances to endothelial cells
    computed_coords_list = []
    for n_closest_num in n_closest_list:
        closest_EC_cells_dict_subset = {x:(y[COORD_IDX][:n_closest_num],y[DIST_IDX][:n_closest_num]) for x,y in closest_EC_cells_dict.items()}
        avg_dist_error = get_avg_dist_error(other_cell_coords, computed_coords)

    avg_dist_error = get_avg_dist_error(other_cell_coords, computed_coords)


In [106]:
from typing import Dict, Any
import hashlib
import json

def dict_hash(dictionary: Dict[str, Any]) -> str:
    """MD5 hash of a dictionary."""
    dhash = hashlib.md5()
    # We need to sort arguments so {'a': 1, 'b': 2} is
    # the same as {'b': 2, 'a': 1}
    encoded = json.dumps(dictionary, sort_keys=True).encode()
    dhash.update(encoded)
    return dhash.hexdigest()

In [161]:
resp = run_simulation(n_EC=1000,
               n_other_cells=100,
               n_dim=3,
               n_closest=[3],
               coord_min = -1,
               coord_max = 1)

In [183]:
closest_EC_cells_3[0][0][:3]

array([118, 979, 311])

In [179]:
[0,2,3,4,4][:3]

[0, 2, 3]

In [140]:
hash((np.hstack((EC_coords, other_cell_coords))).tobytes())

-2396457274118290513

In [167]:
#Generating Cell Coordinates
EC_coords = sim_coordinates(1000, 3, -1, 1)
other_cell_coords = sim_coordinates(10000, 3, -1, 1)

#Computing Closest Neighbors to Each Cell
closest_EC_cells_3 = {x:get_closest_points(EC_coords,other_cell_coords[x], 1000) for x in list(range(len(other_cell_coords)))}

In [168]:
closest_EC_cells_3[0]

(array([118, 979, 311,  28, 383, 912, 815, 959, 985, 375,  47, 346, 610,
        719, 547,  46,  10, 362, 370, 417, 570, 758, 653,  24, 972,  23,
        671,  66, 457, 127, 939, 802, 980, 905, 165,  50, 195, 548, 861,
        870, 339, 343, 625,  17, 924, 431, 893, 849, 327, 934, 317, 799,
         89,  92, 476, 760, 803, 842, 712, 545, 214, 356, 401, 779, 501,
        933, 409, 656, 742, 427, 752, 300,  43, 753, 452, 876, 105, 602,
        527, 146, 890, 832, 553, 560, 761, 885, 407, 286, 633, 860, 620,
        836, 155, 133, 884, 613, 728, 180, 571, 871, 952, 643, 378, 735,
        634, 695, 999, 134, 510,  65, 148, 675, 497, 998, 906, 929,  13,
        528,  15,  53, 581, 413, 516, 917, 411, 416, 273, 563, 679, 647,
        654, 491, 838,  29, 766, 883, 850, 194, 853,  39, 635, 358, 833,
        935, 822, 137, 301,  76, 493, 359,   0, 579, 402,  75,  32,  26,
        556, 992, 477, 326,  36, 429,  63, 851, 349, 275, 572, 823, 216,
        968, 796, 987, 307, 726, 805, 751, 966, 104

In [78]:
resp = estimate_coords(EC_coords, closest_EC_cells_3)

In [73]:
resp[3]

array([ 0.11200023, -0.57408374, -0.50904867])

In [76]:
np.mean(get_dist_error(other_cell_coords, resp))

0.08377276812926232

In [79]:
np.mean(get_dist_error(other_cell_coords, resp))

0.0010995649831437569

In [55]:
other_cell_coords[0]

array([ 0.09814272, -0.03299581,  0.59333181])

In [57]:
EC_coords[closest_EC_cells_3[0]]

array([[-0.40882984,  0.67813756, -0.94476455],
       [-0.62338338,  0.6155872 , -0.92901035],
       [-0.50300839,  0.75957002, -0.81990308],
       [-0.63502526,  0.71578079, -0.87368287]])

In [47]:
import localization as lx

P=lx.Project(mode='2D',solver='LSE')
P.add_anchor('anchorA',(0,1))
P.add_anchor('anchorB',(0,1))

t,label=P.add_target()


t.add_measure('anchorA',50)
t.add_measure('anchorB',0)


P.solve()

# Then the target location is:

print(t.loc)

LSE Geolocating...
p(17.677667328562006,18.677667328562006,0.0)


In [48]:
t

In [60]:
b = t.loc

In [63]:
b.z

0.0